In [1]:
import csv

In [2]:
import random

In [3]:
import math

In [4]:
import copy

In [5]:
def loadCsv(filename):
	lines = csv.reader(open(r'diabetes.csv'))
	dataset = list(lines)
	classes=copy.copy(dataset[0:1])
	dataset=copy.copy(dataset[1:])
	
	for i in range(len(dataset)):
		dataset[i] = [float(x) for x in dataset[i]]
	return dataset

In [6]:
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]
 

In [7]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


In [8]:
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 #Desvio padrão 
def stdev(numbers): 
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

In [29]:
def stdev(numbers):
    print("Numbers",numbers,"\n")
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [28]:
def summarize(dataset):
#     print(len(dataset))
    print("DataSet",*dataset,"\n")
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
#     print([(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)])
    del summaries[-1]
    return summaries

In [13]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    
    summaries = {}
    for classValue, instances in separated.items():
        
#         print("Type",type(instances),"classValue",classValue,"\n")
#         print("Tamanho",len(instances) ,instances,"\n")
        print("SN",summarize(instances),"\n")
        summaries[classValue] = summarize(instances)
    
#     print("\nTamanho",len(summaries) ,summaries)
    print(summaries)
    return summaries

In [14]:
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [15]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
#         print("Tamanho", len(classSummaries),classSummaries,'\n')
        
#         print(inputVector, "Tamanho", len(inputVector))
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    
    return  probabilities


In [16]:
def predict(summaries, inputVector):
    calculateClassProbabilities(summaries, inputVector)
    #probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
            
    return bestLabel

In [17]:
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
#	print(predictions,"\n")
	return predictions

In [18]:
def getAccuracy(testSet, predictions):
	correct = 0
	valorA=0
	valorB=0
	status=-1
	situacao=-1

	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct+=1
			if testSet[i][-1] ==1.0:
				valorA+=1
			elif testSet[i][-1] == 0.0:
				valorB+=1
	print(correct,"A",valorA,"B",valorB)
	if valorA>valorB:
		situacao=(valorA/float(correct)) * 100.0
		status=1
	else:
		situacao=(valorB/float(correct)) * 100.0
		status=0
		
	#Buscando qual a classe teve o maior acerto, acredito que estou no caminho certo, agora e fazer o teste final 
	accuracy=(correct/float(len(testSet))) * 100.0

	return accuracy,status,situacao

In [19]:
def main():
	filename = 'diabetes.csv'
	splitRatio = 0.67
	status=-1
	situacao=-1
	dataset = loadCsv(filename)
	trainingSet, testSet = splitDataset(dataset, splitRatio)
	print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
	# prepare model
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy,status,situacao = getAccuracy(testSet, predictions)
	print('Accuracy: {0}%'.format(accuracy))
	if status==0:
		result="Não teve diabetes" 
	else:
		result="Teve diabetes"
	print("S",result,"{0}%".format(situacao))

In [30]:
main()

Split 768 rows into train=514 and test=254 rows
DataSet [3.0, 111.0, 56.0, 39.0, 0.0, 30.1, 0.557, 30.0, 0.0] [0.0, 102.0, 64.0, 46.0, 78.0, 40.6, 0.496, 21.0, 0.0] [3.0, 96.0, 56.0, 34.0, 115.0, 24.7, 0.944, 39.0, 0.0] [0.0, 106.0, 70.0, 37.0, 148.0, 39.4, 0.605, 22.0, 0.0] [3.0, 78.0, 70.0, 0.0, 0.0, 32.5, 0.27, 39.0, 0.0] [5.0, 117.0, 86.0, 30.0, 105.0, 39.1, 0.251, 42.0, 0.0] [3.0, 126.0, 88.0, 41.0, 235.0, 39.3, 0.704, 27.0, 0.0] [1.0, 79.0, 75.0, 30.0, 0.0, 32.0, 0.396, 22.0, 0.0] [10.0, 75.0, 82.0, 0.0, 0.0, 33.3, 0.263, 38.0, 0.0] [0.0, 137.0, 68.0, 14.0, 148.0, 24.8, 0.143, 21.0, 0.0] [4.0, 97.0, 60.0, 23.0, 0.0, 28.2, 0.443, 22.0, 0.0] [1.0, 153.0, 82.0, 42.0, 485.0, 40.6, 0.687, 23.0, 0.0] [1.0, 79.0, 80.0, 25.0, 37.0, 25.4, 0.583, 22.0, 0.0] [2.0, 84.0, 0.0, 0.0, 0.0, 0.0, 0.304, 21.0, 0.0] [2.0, 100.0, 54.0, 28.0, 105.0, 37.8, 0.498, 24.0, 0.0] [1.0, 73.0, 50.0, 10.0, 0.0, 23.0, 0.248, 21.0, 0.0] [4.0, 131.0, 68.0, 21.0, 166.0, 33.1, 0.16, 28.0, 0.0] [1.0, 100.0, 66.0, 29.

NameError: name 'probabilities' is not defined

In [ ]:
 #probabilities[classValue] = 1
		#for i in range(len(classSummaries)):
		#	mean, stdev = classSummaries[i]
		#	x = inputVector[i]
		#	probabilities[classValue] *= calculateProbability(x, mean, stdev)
	
	#return probabilities